In [1]:
import re
import spacy
import numpy as np
import pandas as pd
from textblob import TextBlob
from bertopic import BERTopic
from transformers import pipeline

In [2]:
all_tweets_to_analyze = pd.read_csv('all_tweets_to_analyze_20250528.csv')

In [3]:
all_tweets_to_analyze.head(3)

,index,politician,date,tweet_text,sentiment_score,subjectivity_score,fear_score,economic_term_count,policy_term_count,currency_mention,economic_term_ratio,named_entities,topic_id,sentiment_volatility_7d
0,9153,Trump,2019-01-01 00:40:26,MEXICO IS PAYING FOR THE WALL through the many...,0.693526,0.555303,0.0,2,0,0,0.039216,"['MEXICO', 'U.S.A.', 'USMCA', 'NAFTA Trade Dea...",0,NaN
1,9152,Trump,2019-01-01 00:51:43,"The Democrats will probably submit a Bill, bei...",-0.889143,1.000000,0.0,0,0,0,0.000000,"['Border Security', 'Tech']",0,1.119116
2,9151,Trump,2019-01-01 01:05:39,...Remember this. Throughout the ages some thi...,-0.970347,0.433333,0.0,0,0,0,0.000000,"['NEVER', 'NEVER', 'ALWAYS', 'ALWAYS', 'NEVER']",0,0.938075


In [4]:
all_tweets_to_analyze.drop(columns=['index'], inplace=True)

In [5]:
all_tweets_to_analyze.columns

Index(['politician', 'date', 'tweet_text', 'sentiment_score',
       'subjectivity_score', 'fear_score', 'economic_term_count',
       'policy_term_count', 'currency_mention', 'economic_term_ratio',
       'named_entities', 'topic_id', 'sentiment_volatility_7d'],
      dtype='object')

In [6]:
all_tweets_to_analyze.shape

(13923, 13)

In [7]:
usd_try_df = pd.read_csv('FX_USDTRY.csv')

In [8]:
usd_try_df.columns

Index(['time', 'open', 'high', 'low', 'close'], dtype='object')

In [9]:
usd_try_df.head()

,time,open,high,low,close
0,2018-02-02T13:00:00+03:00,3.75601,3.75878,3.74659,3.75059
1,2018-02-02T15:00:00+03:00,3.75059,3.76474,3.74460,3.76139
2,2018-02-02T17:00:00+03:00,3.76139,3.77405,3.75539,3.76401
3,2018-02-02T19:00:00+03:00,3.76401,3.76582,3.75669,3.75876
4,2018-02-02T21:00:00+03:00,3.75876,3.77132,3.75155,3.77087


In [10]:
euro_usd_df = pd.read_csv('OANDA_EURUSD.csv')

In [11]:
euro_usd_df.head()

,time,open,high,low,close
0,2018-09-26T22:00:00+03:00,1.17603,1.17716,1.17363,1.17403
1,2018-09-27T00:00:00+03:00,1.17403,1.17524,1.17378,1.17490
2,2018-09-27T02:00:00+03:00,1.17490,1.17574,1.17460,1.17539
3,2018-09-27T04:00:00+03:00,1.17539,1.17560,1.17400,1.17464
4,2018-09-27T06:00:00+03:00,1.17464,1.17531,1.17458,1.17490


In [12]:
xau_usd_df = pd.read_csv('OANDA_XAUUSD.csv')

In [13]:
xau_usd_df.head()

,time,open,high,low,close
0,2018-09-05T04:00:00+03:00,1192.381,1194.674,1191.570,1194.522
1,2018-09-05T06:00:00+03:00,1194.522,1194.522,1193.128,1193.465
2,2018-09-05T08:00:00+03:00,1193.465,1193.868,1191.946,1192.660
3,2018-09-05T10:00:00+03:00,1192.660,1195.926,1191.994,1194.817
4,2018-09-05T12:00:00+03:00,1194.817,1195.590,1193.170,1194.789


In [14]:
sp500_df = pd.read_csv('SP_SPX.csv')

In [15]:
sp500_df.head()

,time,open,high,low,close
0,2018-03-07T20:30:00+03:00,2706.44,2712.82,2702.38,2712.82
1,2018-03-07T21:30:00+03:00,2713.05,2717.60,2710.52,2714.18
2,2018-03-07T22:30:00+03:00,2714.17,2725.51,2714.13,2724.43
3,2018-03-07T23:30:00+03:00,2724.43,2730.60,2724.43,2726.80
4,2018-03-08T17:30:00+03:00,2732.75,2737.29,2727.71,2730.38


In [ ]:
# 1) Make sure both timestamp columns are datetime, drop tz info on rates
all_tweets_to_analyze['date'] = pd.to_datetime(all_tweets_to_analyze['date'])
usd_try_df['time'] = (
    pd.to_datetime(usd_try_df['time'])
      .dt.tz_localize(None)
)

# 2) Sort both DataFrames by their timestamp columns
tweets_sorted = all_tweets_to_analyze.sort_values('date')
rates_sorted  = usd_try_df.sort_values('time')

# 3) Forward‐asof merge to get the closest later rate at tweet‐time
merged = pd.merge_asof(
    left      = tweets_sorted,
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date',
    right_on  = 'time',
    direction = 'forward'
)

# 4) Rename the initial rate and drop the helper time column
merged = (
    merged
      .rename(columns={'close': 'usd_try_ratio'})
      .drop(columns=['time'])
)

# 5) Create helper columns for 2h and 4h later timestamps
merged['date_2h'] = merged['date'] + pd.Timedelta(hours=2)
merged['date_4h'] = merged['date'] + pd.Timedelta(hours=4)

# 6) Forward‐asof merge for the 2h‐later rate
m2 = pd.merge_asof(
    left      = merged.sort_values('date_2h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_2h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'usd_try_2h_later_ratio'})[['usd_try_2h_later_ratio']]

# 7) Forward‐asof merge for the 4h‐later rate
m4 = pd.merge_asof(
    left      = merged.sort_values('date_4h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_4h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'usd_try_4h_later_ratio'})[['usd_try_4h_later_ratio']]

# 8) Attach the new rate columns back to merged, drop helper columns, and restore original order
merged = (
    merged
      .assign(
          usd_try_2h_later_ratio = m2['usd_try_2h_later_ratio'].values,
          usd_try_4h_later_ratio = m4['usd_try_4h_later_ratio'].values
      )
      .drop(columns=['date_2h', 'date_4h'])
      .sort_index()
)

# 9) Overwrite your original DataFrame
all_tweets_to_analyze = merged

# Now all_tweets_to_analyze contains:
#   - usd_try_ratio
#   - usd_try_2h_later_ratio
#   - usd_try_4h_later_ratio

In [19]:
all_tweets_to_analyze.tail()

,politician,date,tweet_text,sentiment_score,subjectivity_score,fear_score,economic_term_count,policy_term_count,currency_mention,economic_term_ratio,named_entities,topic_id,sentiment_volatility_7d,usd_try_ratio,usd_try_2h_later_ratio,usd_try_4h_later_ratio
13918,Biden,2020-10-31 23:42:00,I want to extend my prayers and condolences to...,-0.555184,0.767857,0.0,0,0,0,0.0,"['the Chaldean Assyrian Community', 'Baghdad']",-1,0.879398,8.3572,8.3572,8.3572
13919,Trump,2020-10-31 23:47:53,"Thank you Butler, Pennsylvania!\nhttps://t.co/...",0.988103,0.000000,0.0,0,0,0,0.0,['Pennsylvania'],20,0.879373,8.3572,8.3572,8.3572
13920,Trump,2020-10-31 23:51:29,For the last 4 years you have seen me fight fo...,0.984206,0.022222,0.0,0,0,0,0.0,"['the Republican Party', 'VOTE']",-1,0.879331,8.3572,8.3572,8.3572
13921,Trump,2020-10-31 23:52:33,"Over the next 4 years, we will make America in...",0.960022,0.166667,0.0,0,0,0,0.0,"['America', 'the Manufacturing Superpower', 'C...",-1,0.879218,8.3572,8.3572,8.3572
13922,Biden,2020-10-31 23:55:00,Happy Halloween! https://t.co/i2BT7k0xdI,0.993051,1.000000,0.0,0,0,0,0.0,[],139,0.878574,8.3572,8.3572,8.3572


In [ ]:
# 1) Make sure both timestamp columns are datetime, drop tz info on rates
all_tweets_to_analyze['date'] = pd.to_datetime(all_tweets_to_analyze['date'])
euro_usd_df['time'] = (
    pd.to_datetime(euro_usd_df['time'])
      .dt.tz_localize(None)
)

# 2) Sort both DataFrames by their timestamp columns
tweets_sorted = all_tweets_to_analyze.sort_values('date')
rates_sorted  = euro_usd_df.sort_values('time')

# 3) Forward‐asof merge to get the closest later rate at tweet‐time
merged = pd.merge_asof(
    left      = tweets_sorted,
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date',
    right_on  = 'time',
    direction = 'forward'
)

# 4) Rename the initial rate and drop the helper time column
merged = (
    merged
      .rename(columns={'close': 'euro_usd_ratio'})
      .drop(columns=['time'])
)

# 5) Create helper columns for 2h and 4h later timestamps
merged['date_2h'] = merged['date'] + pd.Timedelta(hours=2)
merged['date_4h'] = merged['date'] + pd.Timedelta(hours=4)

# 6) Forward‐asof merge for the 2h‐later rate
m2 = pd.merge_asof(
    left      = merged.sort_values('date_2h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_2h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'euro_usd_2h_later_ratio'})[['euro_usd_2h_later_ratio']]

# 7) Forward‐asof merge for the 4h‐later rate
m4 = pd.merge_asof(
    left      = merged.sort_values('date_4h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_4h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'euro_usd_4h_later_ratio'})[['euro_usd_4h_later_ratio']]

# 8) Attach the new rate columns back to merged, drop helper columns, and restore original order
merged = (
    merged
      .assign(
          euro_usd_2h_later_ratio = m2['euro_usd_2h_later_ratio'].values,
          euro_usd_4h_later_ratio = m4['euro_usd_4h_later_ratio'].values
      )
      .drop(columns=['date_2h', 'date_4h'])
      .sort_index()
)

# 9) Overwrite your original DataFrame
all_tweets_to_analyze = merged

In [23]:
all_tweets_to_analyze.head(3)

,politician,date,tweet_text,sentiment_score,subjectivity_score,fear_score,economic_term_count,policy_term_count,currency_mention,economic_term_ratio,named_entities,topic_id,sentiment_volatility_7d,usd_try_ratio,usd_try_2h_later_ratio,usd_try_4h_later_ratio,euro_usd_ratio,euro_usd_2h_later_ratio,euro_usd_4h_later_ratio
0,Trump,2019-01-01 00:40:26,MEXICO IS PAYING FOR THE WALL through the many...,0.693526,0.555303,0.0,2,0,0,0.039216,"['MEXICO', 'U.S.A.', 'USMCA', 'NAFTA Trade Dea...",0,NaN,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616
1,Trump,2019-01-01 00:51:43,"The Democrats will probably submit a Bill, bei...",-0.889143,1.000000,0.0,0,0,0,0.000000,"['Border Security', 'Tech']",0,1.119116,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616
2,Trump,2019-01-01 01:05:39,...Remember this. Throughout the ages some thi...,-0.970347,0.433333,0.0,0,0,0,0.000000,"['NEVER', 'NEVER', 'ALWAYS', 'ALWAYS', 'NEVER']",0,0.938075,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616


In [ ]:
# 1) Make sure both timestamp columns are datetime, drop tz info on rates
all_tweets_to_analyze['date'] = pd.to_datetime(all_tweets_to_analyze['date'])
xau_usd_df['time'] = (
    pd.to_datetime(xau_usd_df['time'])
      .dt.tz_localize(None)
)

# 2) Sort both DataFrames by their timestamp columns
tweets_sorted = all_tweets_to_analyze.sort_values('date')
rates_sorted  = xau_usd_df.sort_values('time')

# 3) Forward‐asof merge to get the closest later rate at tweet‐time
merged = pd.merge_asof(
    left      = tweets_sorted,
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date',
    right_on  = 'time',
    direction = 'forward'
)

# 4) Rename the initial rate and drop the helper time column
merged = (
    merged
      .rename(columns={'close': 'xau_usd_ratio'})
      .drop(columns=['time'])
)

# 5) Create helper columns for 2h and 4h later timestamps
merged['date_2h'] = merged['date'] + pd.Timedelta(hours=2)
merged['date_4h'] = merged['date'] + pd.Timedelta(hours=4)

# 6) Forward‐asof merge for the 2h‐later rate
m2 = pd.merge_asof(
    left      = merged.sort_values('date_2h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_2h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'xau_usd_2h_later_ratio'})[['xau_usd_2h_later_ratio']]

# 7) Forward‐asof merge for the 4h‐later rate
m4 = pd.merge_asof(
    left      = merged.sort_values('date_4h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_4h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'xau_usd_4h_later_ratio'})[['xau_usd_4h_later_ratio']]

# 8) Attach the new rate columns back to merged, drop helper columns, and restore original order
merged = (
    merged
      .assign(
          xau_usd_2h_later_ratio = m2['xau_usd_2h_later_ratio'].values,
          xau_usd_4h_later_ratio = m4['xau_usd_4h_later_ratio'].values
      )
      .drop(columns=['date_2h', 'date_4h'])
      .sort_index()
)

# 9) Overwrite your original DataFrame
all_tweets_to_analyze = merged

In [25]:
all_tweets_to_analyze.head(3)

,politician,date,tweet_text,sentiment_score,subjectivity_score,fear_score,economic_term_count,policy_term_count,currency_mention,economic_term_ratio,...,sentiment_volatility_7d,usd_try_ratio,usd_try_2h_later_ratio,usd_try_4h_later_ratio,euro_usd_ratio,euro_usd_2h_later_ratio,euro_usd_4h_later_ratio,xau_usd_ratio,xau_usd_2h_later_ratio,xau_usd_4h_later_ratio
0,Trump,2019-01-01 00:40:26,MEXICO IS PAYING FOR THE WALL through the many...,0.693526,0.555303,0.0,2,0,0,0.039216,...,NaN,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31
1,Trump,2019-01-01 00:51:43,"The Democrats will probably submit a Bill, bei...",-0.889143,1.000000,0.0,0,0,0,0.000000,...,1.119116,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31
2,Trump,2019-01-01 01:05:39,...Remember this. Throughout the ages some thi...,-0.970347,0.433333,0.0,0,0,0,0.000000,...,0.938075,5.2869,5.2869,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31


In [26]:
# 1) Make sure both timestamp columns are datetime, drop tz info on rates
all_tweets_to_analyze['date'] = pd.to_datetime(all_tweets_to_analyze['date'])
sp500_df['time'] = (
    pd.to_datetime(sp500_df['time'])
      .dt.tz_localize(None)
)

# 2) Sort both DataFrames by their timestamp columns
tweets_sorted = all_tweets_to_analyze.sort_values('date')
rates_sorted  = sp500_df.sort_values('time')

# 3) Forward‐asof merge to get the closest later rate at tweet‐time
merged = pd.merge_asof(
    left      = tweets_sorted,
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date',
    right_on  = 'time',
    direction = 'forward'
)

# 4) Rename the initial rate and drop the helper time column
merged = (
    merged
      .rename(columns={'close': 'sp500_ratio'})
      .drop(columns=['time'])
)

# 5) Create helper columns for 2h and 4h later timestamps
merged['date_2h'] = merged['date'] + pd.Timedelta(hours=2)
merged['date_4h'] = merged['date'] + pd.Timedelta(hours=4)

# 6) Forward‐asof merge for the 2h‐later rate
m2 = pd.merge_asof(
    left      = merged.sort_values('date_2h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_2h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'sp500_2h_later_ratio'})[['sp500_2h_later_ratio']]

# 7) Forward‐asof merge for the 4h‐later rate
m4 = pd.merge_asof(
    left      = merged.sort_values('date_4h'),
    right     = rates_sorted[['time', 'close']],
    left_on   = 'date_4h',
    right_on  = 'time',
    direction = 'forward'
).rename(columns={'close': 'sp500_4h_later_ratio'})[['sp500_4h_later_ratio']]

# 8) Attach the new rate columns back to merged, drop helper columns, and restore original order
merged = (
    merged
      .assign(
          sp500_2h_later_ratio = m2['sp500_2h_later_ratio'].values,
          sp500_4h_later_ratio = m4['sp500_4h_later_ratio'].values
      )
      .drop(columns=['date_2h', 'date_4h'])
      .sort_index()
)

# 9) Overwrite your original DataFrame
all_tweets_to_analyze = merged

In [27]:
all_tweets_to_analyze.head(3)

,politician,date,tweet_text,sentiment_score,subjectivity_score,fear_score,economic_term_count,policy_term_count,currency_mention,economic_term_ratio,...,usd_try_4h_later_ratio,euro_usd_ratio,euro_usd_2h_later_ratio,euro_usd_4h_later_ratio,xau_usd_ratio,xau_usd_2h_later_ratio,xau_usd_4h_later_ratio,sp500_ratio,sp500_2h_later_ratio,sp500_4h_later_ratio
0,Trump,2019-01-01 00:40:26,MEXICO IS PAYING FOR THE WALL through the many...,0.693526,0.555303,0.0,2,0,0,0.039216,...,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31,2485.76,2485.76,2485.76
1,Trump,2019-01-01 00:51:43,"The Democrats will probably submit a Bill, bei...",-0.889143,1.000000,0.0,0,0,0,0.000000,...,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31,2485.76,2485.76,2485.76
2,Trump,2019-01-01 01:05:39,...Remember this. Throughout the ages some thi...,-0.970347,0.433333,0.0,0,0,0,0.000000,...,5.2869,1.14616,1.14616,1.14616,1281.31,1281.31,1281.31,2485.76,2485.76,2485.76


In [28]:
all_tweets_to_analyze.to_csv('all_tweets_to_analyze_with_fx_20250528_2331.csv', index=False)